In [2]:
##we build a Hybrid Recommender:
## Segment + Risk + Behaviour → Offer


In [3]:
import pandas as pd

ci = pd.read_csv("../../../data/processed/customer_intelligence.csv")

ci.head()

,segment,churn_probability,risk_level,Total_Trans_Amt,Total_Trans_Ct,Avg_Utilization_Ratio,Total_Relationship_Count
0,Low Engagement Users,0.186830,High Risk,1144,42,0.061,5
1,Low Engagement Users,0.028312,Low Risk,1291,33,0.105,6
2,Low Engagement Users,0.006571,Low Risk,1887,20,0.000,4
3,Credit Revolvers,0.034279,Low Risk,1171,20,0.760,3
4,Low Engagement Users,0.006455,Low Risk,816,28,0.000,5


In [4]:
offers = {
    "cashback_dining": "20% cashback on dining",
    "cashback_travel": "15% cashback on travel",
    "cashback_groceries": "10% cashback on groceries",
    "fuel_offer": "10% cashback on fuel",
    "emi_offer": "No-cost EMI for 3 months",
    "fee_waiver": "Annual fee waiver",
    "credit_limit_increase": "Pre-approved credit limit increase",
    "bonus_rewards": "5x reward points on all purchases"
}


In [5]:
## creating behvious flags
# spending level
ci['spend_level'] = pd.qcut(ci['Total_Trans_Amt'], q=3,
                            labels=['Low Spend','Medium Spend','High Spend'])

# transaction level
ci['txn_level'] = pd.qcut(ci['Total_Trans_Ct'], q=3,
                          labels=['Low Activity','Medium Activity','High Activity'])


In [6]:
def recommend_offer(row):

    # HIGH RISK → retention offers
    if row['risk_level'] == "High Risk":
        
        if row['segment'] == "Low Engagement Users":
            return "Annual fee waiver + bonus reward points"
        
        if row['segment'] == "Credit Revolvers":
            return "No-cost EMI + balance transfer offer"
        
        if row['segment'] == "High Value Spenders":
            return "Premium travel cashback + lounge access"

    # MEDIUM RISK → engagement offers
    if row['risk_level'] == "Medium Risk":
        
        if row['spend_level'] == "High Spend":
            return "5x reward points on all purchases"
        
        if row['txn_level'] == "Low Activity":
            return "10% cashback on groceries & fuel"
        
        return "15% cashback on dining"

    # LOW RISK → upsell / loyalty offers
    if row['risk_level'] == "Low Risk":
        
        if row['segment'] == "High Value Spenders":
            return "Pre-approved credit limit increase"
        
        if row['segment'] == "Credit Revolvers":
            return "Lower interest rate upgrade"
        
        return "Bonus reward points booster"


In [7]:
ci['recommended_offer'] = ci.apply(recommend_offer, axis=1)
ci.head()


,segment,churn_probability,risk_level,Total_Trans_Amt,Total_Trans_Ct,Avg_Utilization_Ratio,Total_Relationship_Count,spend_level,txn_level,recommended_offer
0,Low Engagement Users,0.186830,High Risk,1144,42,0.061,5,Low Spend,Low Activity,Annual fee waiver + bonus reward points
1,Low Engagement Users,0.028312,Low Risk,1291,33,0.105,6,Low Spend,Low Activity,Bonus reward points booster
2,Low Engagement Users,0.006571,Low Risk,1887,20,0.000,4,Low Spend,Low Activity,Bonus reward points booster
3,Credit Revolvers,0.034279,Low Risk,1171,20,0.760,3,Low Spend,Low Activity,Lower interest rate upgrade
4,Low Engagement Users,0.006455,Low Risk,816,28,0.000,5,Low Spend,Low Activity,Bonus reward points booster


In [8]:
"""
Hybrid Recommendation Engine

Offers are generated using:
• Customer segment
• Churn risk level
• Spending behaviour
• Transaction activity

This creates personalized retention and upsell offers.
"""


'\nHybrid Recommendation Engine\n\nOffers are generated using:\n• Customer segment\n• Churn risk level\n• Spending behaviour\n• Transaction activity\n\nThis creates personalized retention and upsell offers.\n'

In [9]:
ci.to_csv("../../../data/processed/customer_offers.csv", index=False)